In [1]:
import dask.dataframe as dd
import dask_geopandas as dg

import pandas as pd
import geopandas as gpd
from siuba import *

import shared_utils

import gcsfs
fs = gcsfs.GCSFileSystem()

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
analysis_date = shared_utils.rt_dates.DATES['oct2022']

In [3]:
analysis_date

'2022-10-12'

In [12]:
bbb = 'gs://calitp-analytics-data/data-analyses/dask_test/speeds_by_operator/speeds_300_2022-10-12.parquet'

In [13]:
bbb_test = pd.read_parquet(bbb)

In [14]:
bbb_test >> head(3)

,calitp_itp_id,trip_id,route_dir_identifier,segment_sequence,min_time,max_time,min_dist,max_dist,meters_elapsed,sec_elapsed,speed_mph
0,300,886311,1211914915,0,2022-10-12 05:10:53,2022-10-12 05:13:51,320.993700,474.533425,153.539726,178.0,1.929598
1,300,886366,1211914915,0,2022-10-12 13:33:20,2022-10-12 14:02:37,312.623304,766.547469,453.924165,1757.0,0.577933
2,300,886364,1211914915,0,2022-10-12 14:16:04,2022-10-12 14:20:53,319.716535,904.145313,584.428778,289.0,4.523762


In [15]:
bbb_test >> filter(_.min_dist.isna()) 

,calitp_itp_id,trip_id,route_dir_identifier,segment_sequence,min_time,max_time,min_dist,max_dist,meters_elapsed,sec_elapsed,speed_mph


In [16]:
bbb_test.min_dist.isna().value_counts()

False    18313
Name: min_dist, dtype: int64

In [17]:
bbb_test.max_dist.isna().value_counts()

False    18313
Name: max_dist, dtype: int64

In [10]:
partition_speeds = 'gs://calitp-analytics-data/data-analyses/dask_test/speeds_2022-10-12'

In [11]:
ddf = dd.read_parquet(partition_speeds)

In [20]:
ddf

,calitp_itp_id,trip_id,route_dir_identifier,segment_sequence,min_time,max_time,min_dist,max_dist,meters_elapsed,sec_elapsed,speed_mph
npartitions=8,,,,,,,,,,,
,int64,object,int64,object,datetime64[ns],datetime64[ns],float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...


In [24]:
ddf.isna().min_time.value_counts().compute()

False    2997770
Name: min_time, dtype: int64

In [25]:
ddf.isna().max_time.value_counts().compute()

False    2997770
Name: max_time, dtype: int64

In [27]:
ddf.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 11 entries, calitp_itp_id to speed_mph
dtypes: datetime64[ns](2), object(2), float64(5), int64(2)

In [29]:
ddf.max().compute()

calitp_itp_id                             372
trip_id                 t_888225_b_18260_tn_0
route_dir_identifier               4292940365
segment_sequence                           99
min_time                  2022-10-13 01:59:59
max_time                  2022-10-13 01:59:59
min_dist                               1000.0
max_dist                               1000.0
meters_elapsed                         1000.0
sec_elapsed                           92579.0
speed_mph                         1015.786997
dtype: object

In [30]:
ddf.min().compute()

calitp_itp_id                             4
trip_id                         -1925972613
route_dir_identifier                 255544
segment_sequence                          0
min_time                2022-10-12 00:00:01
max_time                2022-10-12 00:00:01
min_dist                                0.0
max_dist                                0.0
meters_elapsed                          0.0
sec_elapsed                             0.0
speed_mph                               0.0
dtype: object

In [31]:
ddf[ddf.meters_elapsed == 0.0].compute()

,calitp_itp_id,trip_id,route_dir_identifier,segment_sequence,min_time,max_time,min_dist,max_dist,meters_elapsed,sec_elapsed,speed_mph
21,182,56368815,750250299,40,2022-10-12 02:11:53,2022-10-12 03:24:20,367.200156,367.200156,0.0,4347.0,0.0
29,314,t_1995375_b_33395_tn_0,135703609,16,2022-10-12 15:12:25,2022-10-12 15:12:51,12.657005,12.657005,0.0,26.0,0.0
52,182,56368737,750250299,40,2022-10-12 11:33:38,2022-10-12 11:33:38,468.726611,468.726611,0.0,0.0,NaN
66,127,7993455,1169424098,69,2022-10-12 23:03:19,2022-10-12 23:03:19,492.506816,492.506816,0.0,0.0,NaN
67,361,60125,2274799623,7,2022-10-12 18:21:50,2022-10-12 18:21:50,885.204806,885.204806,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2997763,182,10128000290945-JUNE22,3831692572,20,2022-10-12 10:12:54,2022-10-12 10:12:54,1000.000000,1000.000000,0.0,0.0,NaN
2997764,182,10128000290604-JUNE22,3831692572,20,2022-10-12 06:33:40,2022-10-12 06:33:40,1000.000000,1000.000000,0.0,0.0,NaN
2997765,182,10210002931246-JUNE22,1008563139,25,2022-10-12 14:18:44,2022-10-12 14:18:44,113.919028,113.919028,0.0,0.0,NaN
2997767,182,10534001551456-JUNE22,1954369222,5,2022-10-12 15:51:39,2022-10-12 15:51:39,455.034062,455.034062,0.0,0.0,NaN


## Notes

* insufficient data segments aren't min_dist/max_dist nan, they're min_dist == max_dist
* if min_time also == max_time, speed_mph is nan, otherwise speed_mph is 0.0